In [3]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from mtranslate import translate
import http.cookies
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
from time import sleep

In [4]:
MAX_RETRIES = 10

In [5]:
cookies = pd.read_json('cookies.json')

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63',
    'Connection':'close'
          }

In [74]:
sess = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
sess.mount('https://', adapter)
sess.mount('http://', adapter)

In [5]:
for index, cookie in cookies.iterrows():
    sess.cookies.set(cookie['name'], cookie['value'])

In [7]:
site = sess.get('https://s.taobao.com/search?spm=a21wu.241046-global.6977698868.164.4103b6cbbgbVCY&q=%E9%A5%B0%E5%93%81DIY&acm=lb-zebra-241046-2075394.1003.4.1812236&scm=1003.4.lb-zebra-241046-2075394.OTHER_15206383875223_1812236&bcoffset=-3&ntoffset=-3&p4ppushleft=1%2C48&s=132',
                   headers=headers,)

In [8]:
g_page_config = re.search(r"g_page_config = (.*?);\n", site.text)
page_config_json = json.loads(g_page_config.group(1))
page_items = page_config_json['mods']['itemlist']['data']['auctions']

In [9]:
soup = BeautifulSoup(site.content)

In [10]:
def get_items(res):
    g_page_config = re.search(r"g_page_config = (.*?);\n", res.text)
    page_config_json = json.loads(g_page_config.group(1))
    page_items = page_config_json['mods']['itemlist']['data']['auctions']
    result = []
    for each_item in page_items:
        dict1 = dict.fromkeys(('nid','title','detail_url','view_price','view_sales','nick'))
        dict1['nid'] = each_item['nid']
        dict1['title'] = each_item['title']
        dict1['detail_url'] = each_item['detail_url']
        if "https:" not in dict1['detail_url']:
            dict1['detail_url'] = "https:"+each_item['detail_url']
        dict1['view_price'] = each_item['view_price']
        dict1['view_sales'] = each_item['view_sales']
        dict1['nick'] = each_item['nick']
        result.append(dict1)
#         return dict1
    return result[:]

In [41]:
items_list = get_items(site)

In [83]:
images_urls_array = []
description_html_array = []
urls_saved_array = []

In [84]:
for item in items_list[:10]:
    try:
        site1 = sess.get(item['detail_url'], allow_redirects=True)
        try:
            images_urls.append(eval(re.search(r"auctionImages    : ([^\}]*)", site1.text).group(1).strip()))
        except:
            soup1 = BeautifulSoup(site1.content)
            images_urls.append([x['src'].replace('jpg_60x60', 'jpg_500x500') for x in 
                                soup1.findAll('img', {'src':True,}) if "jpg_60x60q90" in x['src']])
            description_html.append(soup1.find('div', {'class':'attributes-list'}))
        print(item['detail_url'])
        urls_saved_array.append(item['detail_url'])
        sleep(2)
    except Exception as ex:
        print(ex)

https://click.simba.taobao.com/cc_im?p=%CA%CE%C6%B7DIY&s=1101015495&k=537&e=5Hq7V1uRzJLSlRVnwQVU8Cz0YZ6FNCjqm6q3lT4hIvmmqH8A4tuR%2BLT%2FetT5llAk0SHv59WZP6nEp395Hdm0ZRdQ%2BFEl49WVErHMwRu8YKtSbZ3m%2BJ48OV%2FrtkWESJ9gLAJDloaWnZIyTZW5kIBHCfR0f9hiGIlTcw87rVCwPFAxdvvBDnq7gO9M58ToULGmhRdqYvm68aBJfy%2B5fL480dc0LAdg83lIlBtFr5s4dZBGw0QJvzk37oAoeP%2B9qh1pAdTp6AFUG1Gne6iUO7iOXDmHz2Gu3aKZ8OWgp4yOPYikAZd75D%2FOocMM0zb3SnXgysrTguWsCc7lWI%2FjG92UTpB%2FpmRG%2FGfPAhw8ep%2F7p%2BtfWpLjLsixtlIst0MmeMQijSKkHapJZWyG4VwHHgfudBGHObX9Ss9mQuvhaIj9kbNoD0SrG7Uedx5JsxTA5Oc%2F0ngaWMUx271Te8uTO%2F9I09Yv2KKKxe8SB7YH7bv0uefmLxkd5Ltpp4I5aTDJdlqIybNMAlq47laiJ7KvMetfWg%3D%3D#detail
https://click.simba.taobao.com/cc_im?p=%CA%CE%C6%B7DIY&s=1101015495&k=557&e=fQPTmhjdi67SlRVnwQVU8Cz0YZ6FNCjqm6q3lT4hIvmmqH8A4tuR%2BLT%2FetT5llAk0SHv59WZP6lkAmQ1idKrJHsRrQjpLi93Zxs8WUqdLz9SbZ3m%2BJ48OV%2FrtkWESJ9gLAJDloaWnZIyTZW5kIBHCfR0f9hiGIlTcw87rVCwPFAxdvvBDnq7gO9M58ToULGmhRdqYvm68aBJfy%2B5fL480SX7zuDg%2FgcM8q%2FSpA34YUWaLbp5oN1Zg44rZMkAoCjQ

In [85]:
images_urls

[['//gd3.alicdn.com/imgextra/i3/2437641482/O1CN01QqHOs21MondhIq7hU_!!2437641482.jpg',
  '//gd2.alicdn.com/imgextra/i2/2437641482/O1CN01LZm1cO1MondSdSz9d_!!2437641482.jpg',
  '//gd2.alicdn.com/imgextra/i2/2437641482/O1CN01h9aD161MondPKtCWk_!!2437641482.jpg',
  '//gd1.alicdn.com/imgextra/i1/2437641482/O1CN01gCOaZI1MondRPed16_!!2437641482.jpg',
  '//gd4.alicdn.com/imgextra/i4/2437641482/O1CN01wV2Nsi1MondMfN2aw_!!2437641482.jpg'],
 ['//img.alicdn.com/imgextra/i3/2208060893914/O1CN01C6qGVc1emeq41b2dG_!!0-item_pic.jpg_500x500q90.jpg',
  '//img.alicdn.com/imgextra/i1/2208060893914/O1CN014kn2fh1emeq6Mfiah_!!2208060893914.jpg_500x500q90.jpg',
  '//img.alicdn.com/imgextra/i3/2208060893914/O1CN01GUSlRb1emeq7G4JtW_!!2208060893914.jpg_500x500q90.jpg',
  '//img.alicdn.com/imgextra/i3/2208060893914/O1CN01Je9iWF1emeq7usxA1_!!2208060893914.jpg_500x500q90.jpg',
  '//img.alicdn.com/imgextra/i2/2208060893914/O1CN01npDqI81emeq3IHJpn_!!2208060893914.jpg_500x500q90.jpg'],
 ['//gd1.alicdn.com/imgextra/i2/9262

In [79]:
soup1.find('div', {'class':'attributes-list'})

NameError: name 'soup1' is not defined

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException

In [2]:
options = Options()
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
dr = webdriver.Chrome(options=options)

In [6]:
dr.set_page_load_timeout(6)

In [7]:
try:
    dr.get('https://s.taobao.com/search?spm=a21wu.241046-global.6977698868.164.4103b6cbbgbVCY&q=%E9%A5%B0%E5%93%81DIY&acm=lb-zebra-241046-2075394.1003.4.1812236&scm=1003.4.lb-zebra-241046-2075394.OTHER_15206383875223_1812236&bcoffset=6&ntoffset=6&p4ppushleft=1%2C48&s=0')
except TimeoutException:
    pass

In [8]:
for index, cookie in cookies.drop(['httpOnly', 'sameSite', 'session', 'storeId', 'id', 'hostOnly', 'expirationDate'], axis=1).iterrows():
    dr.add_cookie(dict(cookie))

In [9]:
url = 'https://s.taobao.com/search?q=camping+tents&type=p&tmhkh5=&spm=a21wu.241046-global.a2227oh.d100&from=sea_1_searchbutton&catId=100'

In [10]:
try:
    dr.get(url)
except TimeoutException:
    pass

In [12]:
NUM_PAGES = 1

In [13]:
prod_urls_array = []
vendors_array = []

In [14]:
for i in range(NUM_PAGES):
    for _ in range(7):
        dr.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        sleep(0.5)
    sleep(1)
    
    soup = BeautifulSoup(dr.page_source)
    prods = soup.find('div', {'id':'mainsrp-itemlist'}).findAll('div', {"data-category": "auctions"})
    prod_urls = [x.find('a')['href'] for x in prods]
    vendors = [x.getText('', True) for x in soup.findAll('div', {'class': 'shop'})]
    prod_urls_array += prod_urls
    vendors_array += vendors
    
    dr.find_element_by_xpath('//li[@class="item next"]').click() 
    WebDriverWait(dr, 10).until(
            EC.visibility_of_element_located((By.XPATH, f'//span[@class="num" and contains(text(),{i+2})]'))
        )

In [15]:
prod_urls_array

['https://click.simba.taobao.com/cc_im?p=camping%20tents&s=1303860205&k=609&e=FAnZnSHnMSvsPHLrh8ZWs64RUMehw5DXhwRystnZPdpDHJDTMvDat2os%2FfsdHB%2B3mNieRdspZX4k4ofcPTVbQaYUEHs%2FwIPkXwcmZ2Pkl%2BtpffWeuVwQ%2BcW3PdKh42uxEt7oEtbJ9HIyfKFpqFWpEhvDWwgCg4lLANqIYdJC75upywDj2dbo9pe5QB1PG1hl7pJYWl3z6pIO2ZRNX%2FzV9n1k3vvvnwrU3e5xeWEHTB7BEAn4qQiUAUvbIfHL9Gom48Ke6B5uQbl7nONxEAZDRv79ae3ohN0dKbcUO1TrYKdFafF5fnpkn7tiQk7RQ3IUh2Nz%2Ftx0szZcF9QDxZo4%2FPl1pZuiEGkirFelArhnD0AqMIsDlZZ0mE1OWiTmk6QpimGUM6YzZ%2B8nRNTIuqQX6T8mSlWEPDsTWd2dxd7LHbDM1bmsrxedmdyhbSMxvzBl%2F9WNpfOi9okkL43O0VYfjEN4lwqu1%2BQWWVervuaLiLQwfqjop6Twx2f%2Bq4otDyTZ1aB%2BBMKW9%2B0EPBZZ4y2OIARxbZYZ8jaSIIAeZlpjlSWGivKhb65BQRhQZgcPgypwG4m%2BdqComR5SfpeOVBzmMl1HKHe0PHFT',
 'https://click.simba.taobao.com/cc_im?p=camping%20tents&s=1303860205&k=621&e=OK9hm9ijJBrsPHLrh8ZWs64RUMehw5DXhwRystnZPdpDHJDTMvDat2os%2FfsdHB%2B3mNieRdspZX60i4019xlSclnvrsiMmWTRXwcmZ2Pkl%2BtpffWeuVwQ%2BcW3PdKh42uxEt7oEtbJ9HIyfKFpqFWpEhvDWwgCg4lLANqIYdJC75upywDj2dbo9pe5QB1PG1hl7pJ

In [16]:
vendors_array

['极冰户外专营店',
 'naturehike途鸿专卖店',
 'fantasygarden旗舰店',
 '莱伊晶旗舰店',
 '玲娇旗舰店',
 '小风筝呀0316',
 'mseav',
 'mseav',
 'hk蒸气士多',
 'pengmin885',
 'freelife1988',
 'tb26304195',
 'freelife1988',
 '淘沙疯丫头',
 '阿节节高',
 '淘沙疯丫头',
 'ellen1816',
 '黄亭予2',
 '八冠军',
 '爱在飞翔7391',
 'wangdongxia042',
 '八冠军',
 'g253034460',
 'g253034460',
 '阿节节高',
 'wangdongxia042',
 '乐尚企业商场',
 '黄亭予2',
 '杨馒头258',
 'pengmin885',
 'sophieliu886',
 '花开雨落百货',
 'szxuanyuan1986',
 'w306807534',
 'w306807534',
 '厨房燃气热水器配件商城',
 'sophieliu886',
 'szxuanyuan1986',
 '微笑生活吧',
 'suishuai1993',
 'ellen1816',
 '刚哥威武霸气233',
 'kunchong630',
 '气克斗牛杂货铺',
 '八龙门',
 '聚优品365',
 'wuyangboya',
 '气克斗牛杂货铺']

In [69]:
for vend_num, url in enumerate(prod_urls_array[:3]):
    if "https" not in url:
        url = "https:" + url
    try:
        dr.get(url)
#         sleep(5)
        
#         try:
#             popup_dialogue = WebDriverWait(dr, 5).until(
#                 EC.presence_of_element_located((By.CLASS_NAME, "baxia-dialog-close"))
#             )
#             popup_dialogue.click()
#             popup_dialogue = WebDriverWait(dr, 5).until(
#                 EC.presence_of_element_located((By.CLASS_NAME, "baxia-dialog-close"))
#             )
#             popup_dialogue.click()
#             print('reached1')
#         except:
#             print('s')
    except TimeoutException:
        sleep(5)
    try:
#         print('reached2')
        WebDriverWait(dr, 15).until(
            EC.presence_of_element_located((By.ID, "J_UlThumb"))
        )
        WebDriverWait(dr, 15).until(
            EC.presence_of_element_located((By.ID, "attributes"))
        )
#         print('reached3')
        WebDriverWait(dr, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tb-detail-hd"))
        )
        WebDriverWait(dr, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tm-price"))
        )
#         print('reache4')
    except:
        sleep(2)
        try:
            WebDriverWait(dr, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "tb-main-title"))
            )
            WebDriverWait(dr, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tb-rmb-num"))
            )
        except Exception as ex:
            print('skipped')
            print(ex)
            continue
            
        
    scrapify(dr.page_source, dr.current_url, vend_num)
    print('scraped\n')
        
#     break

scraped

scraped

scraped



# detail.tmall

In [67]:
name_array, original_price_array, discount_price_array = [], [], []
img_urls_array, description_html_array, description_text_array = [], [], []
vendors_final_array, prod_urls_final_array = [], []

In [68]:
def scrapify(site, url, vend_num):
    soup1 = BeautifulSoup(site)
    if 'detail.tmall.com' in url:
        name = soup1.find('div', {'class':'tb-detail-hd'}).getText('\n', True)
        original_price = soup1.find('span', {'class':'tm-price'}).getText('', True)
        try:
            discount_price = soup1.findAll('span', {'class':'tm-price'})[1].getText('', True)
        except:
            discount_price = ''
        img_urls = [x['src'].replace('400x400', '800x800').replace('jpg_50x50', 'jpg_800x800')
                    for x in soup1.find('ul', {'id': 'J_UlThumb'}).findAll('img', {'src': True})]
        description_html = soup1.find('div', {'id': 'attributes'})
        description_text = soup1.find('div', {'id': 'attributes'}).getText('\n', True)
        vendor = vendors_array[vend_num]
    
    
    elif 'item.taobao.com' in url:
        name = soup1.find('h3', {'class': 'tb-main-title'}).getText('', True)
        try:
            discount_price = soup1.find('em', {'class': "tb-rmb-num", 'id':True}).getText('', True)
        except:
            discount_price = ''
        original_price = soup1.find('em', {'class': "tb-rmb-num", 'id':False}).getText('', True)
            
        img_urls = [x['src'].replace('400x400', '800x800').replace('50x50', '800x800').replace('60x60', '800x800')
                    for x in soup1.find('ul', {'id': 'J_UlThumb'}).findAll('img', {'src': True})]
        description_html = soup1.find('div', {'id': 'attributes'})
        description_text = soup1.find('div', {'id': 'attributes'}).getText('\n', True)
        vendor = vendors_array[vend_num]
        
    else:
        print('weird url => ', url)
        return
    
    name_array.append(name)
    vendors_final_array.append(vendor)
    original_price_array.append(original_price)
    discount_price_array.append(discount_price) 
    img_urls_array.append(img_urls)
    description_html_array.append(description_html)
    description_text_array.append(description_text)
    prod_urls_final_array.append(url)

# item.taobao

In [51]:
name = soup1.find('h3', {'class': 'tb-main-title'}).getText('', True)
discount_price = soup1.find('em', {'class': "tb-rmb-num", 'id':True}).getText('', True)
original_price = soup1.find('em', {'class': "tb-rmb-num", 'id':False}).getText('', True)
img_urls = [x['src'].replace('400x400', '800x800') for x in soup1.find('ul', {'id': 'J_UlThumb'}).findAll('img', {'src': True})]
description_html = soup1.find('div', {'id': 'attributes'})
description_text = soup1.find('div', {'id': 'attributes'}).getText('\n', True)

In [30]:
df = pd.DataFrame({
    'Title': name_array,
    'Vendor': vendors_final_array,
    'OriginalPrice': original_price_array,
    'DiscountPrice': discount_price_array,
    'DescriptionText': description_text_array,
    'DescriptionHTML': description_html_array,
    'ProdLink': prod_urls_final_array,
    'ImgURLs': img_urls_array,
})

In [32]:
df.head()

,Title,Vendor,OriginalPrice,DiscountPrice,DescriptionText,DescriptionHTML,ProdLink,ImgURLs
0,naturehike挪客户外帐篷星河双人2人情侣野营露营超轻15D防暴雨\n定制款15d面料...,极冰户外专营店,1480.00-1880.00,,品牌名称：\nNaturehike\n产品参数：\n品牌: Naturehike\n帐篷大小...,"<div class=""attributes"" id=""attributes""> <div ...",https://detail.tmall.com/item.htm?id=626609585...,[//img.alicdn.com/imgextra/i1/2136921657/O1CN0...
1,Naturehike挪客印第安帐篷户外棉布帐篷露营野营加厚野奢Glamping\n4.8平米...,naturehike途鸿专卖店,3699.00,,品牌名称：\nNaturehike\n产品参数：\n品牌: Naturehike\n产地: ...,"<div class=""attributes"" id=""attributes""> <div ...",https://detail.tmall.com/item.htm?id=638521879...,[//img.alicdn.com/imgextra/i1/3485536181/O1CN0...
2,Fantasy Garden梦花园帐篷户外野营加厚印第安轻奢露营棉布蒙古包\n轻奢系列，防水...,fantasygarden旗舰店,1799.00-2599.00,,品牌名称：\nFANTASY GARDEN\n产品参数：\n品牌: FANTASY GARD...,"<div class=""attributes"" id=""attributes""> <div ...",https://detail.tmall.com/item.htm?id=636703711...,[//img.alicdn.com/imgextra/https://img.alicdn....


In [70]:
df = pd.DataFrame(columns=['Handle', 'Title', 'Description(HTML)', 'Description(text)','OriginalPrice',
                           'DiscountPrice', 'ProductURL', 'Image Src', 'Image Position'
                            ])
#preparing according to shopify import template
for index, imgs in enumerate(img_urls_array):
    for i, src in enumerate(imgs):
        if i== 0:
            df = df.append({
                'Handle': name_array[index],
                'Title': name_array[index],
                'Description(HTML)': description_html_array[index],
                'Description(text)': description_text_array[index],
                'OriginalPrice': original_price_array[index],
                'DiscountPrice': discount_price_array[index],
                'ProductURL': prod_urls_array[index],
                'Image Src': src,
                'Image Position': i+1
            }, ignore_index = True)
        else:
            df = df.append({
                'Handle': name_array[index],
                'Title': '',
                'Description(HTML)': '',
                'Description(text)': '',
                'OriginalPrice': '',
                'DiscountPrice': '',
                'ProductURL': '',
                'Image Src': src,
                'Image Position': i+1
            }, ignore_index = True)
            

In [71]:
df['Image Src'] = df['Image Src'].apply(lambda x: x.replace('60x60', '800x800').replace('400x400', '800x800').replace('50x50', '800x800'))
writer = pd.ExcelWriter(r'file.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer, index=False)
writer.close()

In [77]:
soup1 = BeautifulSoup(dr.page_source)
if 'detail.tmall.com' in url:
    name = soup1.find('div', {'class':'tb-detail-hd'}).getText('\n', True)
    original_price = soup1.find('span', {'class':'tm-price'}).getText('', True)
    try:
        discount_price = soup1.findAll('span', {'class':'tm-price'})[1].getText('', True)
    except:
        discount_price = ''
    img_urls = [x['src'].replace('400x400', '800x800').replace('jpg_50x50', 'jpg_800x800')
                for x in soup1.find('ul', {'id': 'J_UlThumb'}).findAll('img', {'src': True})]
    description_html = soup1.find('div', {'id': 'attributes'})
    description_text = soup1.find('div', {'id': 'attributes'}).getText('\n', True)
    vendor = vendors_array[vend_num]

In [78]:
soup1.findAll('span', {'class':'tm-price'})[0].getText('', True)

'1480.00-1880.00'